In [1]:
# pip install scikit-learn geopandas h3pandas h3~=3.0 -q

In [2]:
import geopandas as gpd
import h3pandas
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [3]:
schools = gpd.read_file("./data/youtube_course/NYC/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("./data/youtube_course/NYC/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("./data/youtube_course/NYC/New York City Bike Routes_20241223.geojson")
neighborhoods = gpd.read_file("https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("./data/youtube_course/NYC/Parks Properties_20241223.geojson")

In [4]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

# Analyze neighborhoods

In [5]:
# Analysis for each neighborhood
def analyze_neighborhood(neighborhood_geometry):
    # Count features intersecting the neighborhood boundary
    num_schools = schools[schools.geometry.intersects(neighborhood_geometry)].shape[0]
    num_subways = subways[subways.geometry.intersects(neighborhood_geometry)].shape[0]
    bike_path_length = bike_paths[bike_paths.geometry.intersects(neighborhood_geometry)].length.sum()
    park_area = parks[parks.geometry.intersects(neighborhood_geometry)].area.sum()

    return num_schools, num_subways, bike_path_length, park_area

In [6]:
# Apply analysis to each neighborhood.
# Calculamos el número de escuelas, entrdas de metro, longitud de carriles bici y áreas de parques para cada vecindario
neighborhoods[['num_schools', 'num_subways', 'bike_path_length', 'park_area']] = neighborhoods.geometry.apply(
    lambda geom: pd.Series(analyze_neighborhood(geom))
)

In [7]:
neighborhoods

,neighborhood,boroughCode,borough,X.id,geometry,num_schools,num_subways,bike_path_length,park_area
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",12.0,5.0,12972.782695,5.191057e+05
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.0,0.0,5886.886135,7.365924e+06
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",1.0,0.0,0.000000,9.533262e+06
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8255425.402 4959593.695, -8255451.0...",0.0,0.0,0.000000,2.440717e+05
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8244408.305 4952520.907, -8244409.6...",2.0,0.0,9014.602134,4.578543e+06
...,...,...,...,...,...,...,...,...,...
307,Woodside,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8226606.801 4976664.013, -8226525.9...",5.0,12.0,15405.289108,5.105127e+04
308,East Williamsburg,3,Brooklyn,None,"POLYGON ((-8231107.865 4971393.424, -8231029.9...",5.0,5.0,7544.864539,6.500018e+04
309,Hamilton Heights,1,Manhattan,None,"POLYGON ((-8232132.004 4988019.867, -8231085.6...",10.0,16.0,7005.241155,1.617397e+06
310,Williamsburg,3,Brooklyn,None,"POLYGON ((-8232919.255 4971877.704, -8232410.0...",28.0,43.0,49675.601087,7.826565e+05


In [8]:
# Normalize results (0 to 1 scale)
scaler = MinMaxScaler()
columns_to_normalize = ['num_schools', 'num_subways', 'bike_path_length', 'park_area']
neighborhoods[columns_to_normalize] = scaler.fit_transform(neighborhoods[columns_to_normalize])

In [9]:
neighborhoods.head(3)

,neighborhood,boroughCode,borough,X.id,geometry,num_schools,num_subways,bike_path_length,park_area
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",0.193548,0.045045,0.181510,0.026226
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.000000,0.000000,0.082367,0.372141
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",0.016129,0.000000,0.000000,0.481639


In [10]:
# Aggregate results using touching neighborhoods
def aggregate_touching_neighborhoods(neighborhood_index):
    current_geometry = neighborhoods.loc[neighborhood_index, 'geometry']
    touching_indices = neighborhoods[neighborhoods.geometry.touches(current_geometry)].index

    if not touching_indices.empty:
        neighbor_values = neighborhoods.loc[touching_indices, columns_to_normalize].mean()
    else:
        neighbor_values = neighborhoods.loc[neighborhood_index, columns_to_normalize]

    return neighbor_values

In [11]:
# Apply aggregation to each neighborhood
neighborhoods[columns_to_normalize] = neighborhoods.index.to_series().apply(
    lambda idx: aggregate_touching_neighborhoods(idx)
)

In [12]:
neighborhoods.head(3)

,neighborhood,boroughCode,borough,X.id,geometry,num_schools,num_subways,bike_path_length,park_area
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",0.048387,0.027027,0.006424,0.024072
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.134409,0.000000,0.141117,0.308764
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",0.029570,0.012012,0.055079,0.202038


In [13]:
# Final normalization (0 to 1 scale)
neighborhoods[columns_to_normalize] = scaler.fit_transform(neighborhoods[columns_to_normalize])

In [14]:
neighborhoods.head(3)

,neighborhood,boroughCode,borough,X.id,geometry,num_schools,num_subways,bike_path_length,park_area
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",0.066667,0.036585,0.009243,0.025121
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.185185,0.000000,0.203034,0.322221
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",0.040741,0.016260,0.079246,0.210843


In [15]:
neighborhoods['index_score'] = neighborhoods['num_schools'] + neighborhoods['num_subways'] + neighborhoods['bike_path_length'] + neighborhoods['park_area'] 

In [16]:
import leafmap

m = leafmap.Map()
m.add_data(
    neighborhoods, column="index_score", scheme="Quantiles", cmap="Blues", legend_title="Index"
)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [20]:
# Save or visualize the results
neighborhoods.to_file("./data/processed_data/neighborhood_access_index.geojson", driver="GeoJSON")
neighborhoods.to_file("./data/processed_data/neighborhood_access_index.shp") # Para guardar como un shapefile

# Set up the H3 Grid

In [29]:
neighborhoods = neighborhoods.to_crs('EPSG:4326')

In [30]:
resolution = 9  # Adjust resolution as needed
gdf_h3 = neighborhoods.h3.polyfill(resolution)

In [ ]:
gdf_h3.head()

In [ ]:
gdf_h3 = neighborhoods.h3.polyfill(resolution, explode=True)
gdf_h3.head()

In [ ]:
gdf_h3 = gdf_h3[gdf_h3['h3_polyfill'].isnull() == False].set_index('h3_polyfill')
gdf_h3.index.name = None
gdf_h3

In [34]:
gdf_h3 = gdf_h3.h3.h3_to_geo_boundary()

In [ ]:
pip install folium matplotlib mapclassify -q

In [ ]:
gdf_h3.explore()

In [37]:
gdf_h3_proj = gdf_h3.to_crs('EPSG:3857')

In [38]:
# Analysis for each hex cell
def analyze_access(hex_geometry):
    # Buffer hex geometry
    buffer_1600m = hex_geometry.buffer(1600)
    buffer_800m = hex_geometry.buffer(800)

    # Count features within buffers
    num_schools = schools[schools.geometry.intersects(buffer_1600m)].shape[0]
    num_subways = subways[subways.geometry.intersects(buffer_1600m)].shape[0]
    bike_path_length = bike_paths[bike_paths.geometry.intersects(buffer_1600m)].length.sum()
    park_area = parks[parks.geometry.intersects(buffer_800m)].area.sum()

    return num_schools, num_subways, bike_path_length, park_area

In [39]:
gdf_h3_proj = gdf_h3.to_crs('EPSG:3857')

In [40]:
gdf_h3_proj[['num_schools', 'num_subways', 'bike_path_length', 'park_area']] = gdf_h3_proj.geometry.apply(
    lambda hex_geom: pd.Series(analyze_access(hex_geom))
)


In [ ]:
gdf_h3_proj.head()

In [42]:
gdf_h3_proj['h3_index'] = gdf_h3_proj.index

# Run the normalization analysis

In [43]:
import h3

In [50]:
# Normalize results
scaler = MinMaxScaler()
normalized_columns = ['num_schools', 'num_subways', 'bike_path_length', 'park_area']
gdf_h3_proj[normalized_columns] = scaler.fit_transform(gdf_h3_proj[normalized_columns])

# Aggregate results using neighboring cells
def aggregate_neighbors(h3_index):
    neighbors = h3.k_ring(h3_index, 2)  # 2-k ring
    neighbor_values = gdf_h3_proj[gdf_h3_proj['h3_index'].isin(neighbors)][normalized_columns].mean()
    return neighbor_values

gdf_h3_proj[normalized_columns] = gdf_h3_proj['h3_index'].apply(
    lambda h3_index: aggregate_neighbors(h3_index)
)

# # Final normalized analysis
gdf_h3_proj[normalized_columns] = scaler.fit_transform(gdf_h3_proj[normalized_columns])

# Save or visualize the results
gdf_h3_proj.to_file("access_index.geojson", driver="GeoJSON")

In [ ]:
gdf_h3_proj.head(3)

# Create the total score

In [52]:
gdf_h3_proj['index_score'] = gdf_h3_proj['num_schools'] + gdf_h3_proj['num_subways'] + gdf_h3_proj['bike_path_length'] + gdf_h3_proj['park_area'] 

In [53]:
import leafmap

In [54]:
gdf_h3_map = gdf_h3_proj.to_crs('EPSG:4326')

In [ ]:
m = leafmap.Map()
m.add_data(
    gdf_h3_map, column="index_score", scheme="Quantiles", cmap="Blues", legend_title="Index"
)
m